In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import scipy

In [22]:
#Load the dataset
pd.set_option('display.max_columns', None)
df = pd.read_csv('C:/Users/Andat/Downloads/university common folder/Project/College-fight-song-DataViz/data/fight-songs.csv')
print(df.shape)

(65, 23)


In [23]:
# Data Cleaning: Convert 'year' to numeric, treat 'Unknown' as NaN
df['year_clean'] = pd.to_numeric(df['year'], errors='coerce')
df.head()

,school,conference,song_name,writers,year,student_writer,official_song,contest,bpm,sec_duration,fight,number_fights,victory,win_won,victory_win_won,rah,nonsense,colors,men,opponents,spelling,trope_count,spotify_id,year_clean
0,Notre Dame,Independent,Victory March,Michael J. Shea and John F. Shea,1908,No,Yes,No,152,64,Yes,1,Yes,Yes,Yes,Yes,No,Yes,Yes,No,No,6,15a3ShKX3XWKzq0lSS48yr,1908.0
1,Baylor,Big 12,Old Fight,Dick Baker and Frank Boggs,1947,Yes,Yes,No,76,99,Yes,4,Yes,Yes,Yes,No,No,Yes,No,No,Yes,5,2ZsaI0Cu4nz8DHfBkPt0Dl,1947.0
2,Iowa State,Big 12,Iowa State Fights,"Jack Barker, Manly Rice, Paul Gnam, Rosalind K...",1930,Yes,Yes,No,155,55,Yes,5,No,No,No,Yes,No,No,Yes,No,Yes,4,3yyfoOXZQCtR6pfRJqu9pl,1930.0
3,Kansas,Big 12,I'm a Jayhawk,"George ""Dumpy"" Bowles",1912,Yes,Yes,No,137,62,No,0,No,No,No,No,Yes,No,Yes,Yes,No,3,0JzbjZgcjugS0dmPjF9R89,1912.0
4,Kansas State,Big 12,Wildcat Victory,Harry E. Erickson,1927,Yes,Yes,No,80,67,Yes,6,Yes,No,Yes,No,No,Yes,No,No,No,3,4xxDK4g1OHhZ44sTFy8Ktm,1927.0


In [24]:
df.columns

Index(['school', 'conference', 'song_name', 'writers', 'year',
       'student_writer', 'official_song', 'contest', 'bpm', 'sec_duration',
       'fight', 'number_fights', 'victory', 'win_won', 'victory_win_won',
       'rah', 'nonsense', 'colors', 'men', 'opponents', 'spelling',
       'trope_count', 'spotify_id', 'year_clean'],
      dtype='object')

In [25]:
# Look at the structure
df.info()
# df.isnull().sum() - there's no missing data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   school           65 non-null     object 
 1   conference       65 non-null     object 
 2   song_name        65 non-null     object 
 3   writers          65 non-null     object 
 4   year             65 non-null     object 
 5   student_writer   65 non-null     object 
 6   official_song    65 non-null     object 
 7   contest          65 non-null     object 
 8   bpm              65 non-null     int64  
 9   sec_duration     65 non-null     int64  
 10  fight            65 non-null     object 
 11  number_fights    65 non-null     int64  
 12  victory          65 non-null     object 
 13  win_won          65 non-null     object 
 14  victory_win_won  65 non-null     object 
 15  rah              65 non-null     object 
 16  nonsense         65 non-null     object 
 17  colors           6

In [26]:
rows = []
features = ['student_writer','official_song','contest','fight','victory','win_won','victory_win_won','rah','nonsense','colors','men','opponents','spelling']

for col in features:
    uniques = df[col].dropna().unique()
    for val in uniques:
        rows.append({"column": col, "category": val})

unique_long_df = pd.DataFrame(rows)
unique_long_df


,column,category
0,student_writer,No
1,student_writer,Yes
2,student_writer,Unknown
3,official_song,Yes
4,official_song,No
5,contest,No
6,contest,Yes
7,fight,Yes
8,fight,No
9,victory,Yes


In [27]:
fs = df.copy()
# List of all trope columns that are "Yes/No"
trope_cols = ['fight', 'victory', 'win_won', 'rah', 'nonsense', 'colors', 'men', 'opponents', 'spelling']

# Convert 'Yes'/'No' to 1/0 for all these columns
for col in trope_cols:
    fs[col] = fs[col].map({'Yes': 1, 'No': 0})

In [28]:

## Custom Metrics for a fight song
# 1. Aggression score [fight, victory, win_won, opponents]
# 2. Spirit score [rah, nonsense, colors, spelling]
# 3. Trope density [how 'busy' is the song]
# 4. Big ten indicator for the challenge requirement

fs['aggression_score'] = fs[['fight', 'victory', 'win_won', 'opponents']].sum(axis=1)
fs['spirit_score'] = fs[['rah', 'nonsense', 'colors', 'spelling']].sum(axis=1)
fs['trope_density'] = fs['trope_count'] / fs['sec_duration']
fs['is_big_ten'] = fs['conference'] == 'Big Ten'

print("New features created!")
fs[['school', 'conference','aggression_score','spirit_score','trope_density']].head()

New features created!


,school,conference,aggression_score,spirit_score,trope_density
0,Notre Dame,Independent,3,2,0.093750
1,Baylor,Big 12,3,2,0.050505
2,Iowa State,Big 12,1,2,0.072727
3,Kansas,Big 12,1,1,0.048387
4,Kansas State,Big 12,2,1,0.044776


In [ ]:
## Tempo Categorization (Feature binning)
fs['tempo_type'] = pd.cut(fs['bpm'], 
                          bins=[0, 100, 140, 200], 
                          labels=['Anthem', 'March', 'Sprint'])

In [30]:
## "Nonsense" Correlation
# Hypothesis: Studens are more likely to write "exurberant nonsense" than professional songwriters

fs.groupby('student_writer')['nonsense'].mean()

student_writer
No         0.066667
Unknown    0.000000
Yes        0.250000
Name: nonsense, dtype: float64

In [31]:
## Big Ten "Uniqueness DNA"

# Calculate the mean (frequency) for Big Ten vs. Others
# We use .groupby() to split the data and [tropes].mean() to get the average occurrence
comparison = fs.groupby('is_big_ten')[trope_cols].mean().reset_index()

# Reshape the data from 'Wide' to 'Long' format (essential for Plotly grouped bars)
# This turns columns into rows so Plotly can color-code them
comparison_melted = comparison.melt(id_vars='is_big_ten', var_name='Trope', value_name='Frequency')

# Map the True/False to readable names
comparison_melted['Group'] = comparison_melted['is_big_ten'].map({True: 'Big Ten', False: 'National Average'})

# 5. Visualize
fig_c = px.bar(comparison_melted, 
             x='Trope', 
             y='Frequency', 
             color='Group', 
             barmode='group',
             title='DNA Comparison: Big Ten vs. National Average',
             color_discrete_map={'Big Ten': '#E00122', 'National Average': '#777777'}) # B10 Red vs Gray

fig_c.show()

The Big Ten leads in 'Spirit' metrics suggesting a culture that prioritizes shared rituals and historical continuity, as compared to national averages..

In [45]:
fs_long = fs.melt(
    id_vars="conference",
    value_vars=["spirit_score", "aggression_score"],
    var_name="metric",
    value_name="score"
)

fig = px.box(
    fs_long,
    x="conference",
    y="score",
    color="conference",
    facet_col="metric",
    points="all",
    notched=True,
    labels={"score": "Gap Score (0–4)", "metric": ""}
)

fig.update_layout(
    title="Spirit vs Aggression Gap by Conference",
    showlegend=False,
    height = 350,
    width = 1000
)

fig.show()


In [39]:
from scipy import stats

# Filter the scores for the two conferences to compare
big_ten_scores = fs[fs['conference'] == 'Big Ten']['spirit_score']
sec_scores = fs[fs['conference'] == 'SEC']['spirit_score']

# Run an Independent T-Test
t_stat, p_val = stats.ttest_ind(big_ten_scores, sec_scores)

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_val:.4f}")

if p_val < 0.05:
    print("Conclusion: The Spirit Gap is Statistically Significant (p < 0.05)")
else:
    print("Conclusion: There is no significant difference; any gap is likely due to random chance.")

T-statistic: 0.8360
P-value: 0.4108
Conclusion: There is no significant difference; any gap is likely due to random chance.


Contrary to public perception, there is no statistically significant difference in lyrical aggression between the SEC and Big Ten ($p > 0.05$).

However, the Big Ten exhibits $20\%$ less variance in song structure, pointing to a more 'standardized' midwestern tradition.

In [50]:
import plotly.express as px

# Create a histogram that shows the "Shape" of the conferences
fig = px.histogram(fs[fs['conference'].isin(['Big Ten', 'SEC'])], 
                   x="aggression_score", 
                   color="conference", 
                   marginal="rug", # Shows individual data points at the bottom
                   barmode="overlay", # Let the bars overlap to see the 'Monolith'
                   nbins=10,
                   title="The Lyrical Monolith: Distribution of Aggression Score")

fig.update_traces(opacity=0.75) # Makes overlapping bars easier to see
fig.show()

In [51]:
# Calculate Variance and Mean
cv_stats = fs.groupby('conference')['aggression_score'].agg(['mean', 'std'])
cv_stats['cv'] = cv_stats['std'] / cv_stats['mean'] # The Coefficient of Variation

print(cv_stats)

                 mean       std        cv
conference                               
ACC          1.357143  1.277446  0.941276
Big 12       1.700000  1.159502  0.682060
Big Ten      1.857143  1.231456  0.663092
Independent  3.000000       NaN       NaN
Pac-12       2.083333  1.164500  0.558960
SEC          2.142857  1.231456  0.574679


Some conclusions on the basis of correlation of variation:

1. Big Ten has one of the highest means but a relatively low CV
    Meaning: Big Ten fight songs are not just "aggressive"; they are consistently aggressive.

2. SEC has higher mean and an even lower CV
    Meaning: SEC is the most standardized conference in this list.

3. This analysis shows that SEC and Big Ten are representing a standardized blueprint of American sports culture. Conversely, the ACC remains a fragmented landscape of musical styles, with variance nearly double that of it its peers.